## **Capstone MDS - XGBoost**

#### **_Integrantes:_**  

1. Evans Díaz   
1. Felipe Ojeda
1. Javier Cofre  
1. Luis Reinoso

In [1]:
# importar librerías
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import statsmodels.api as sm
import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform
from matplotlib.ticker import EngFormatter

In [2]:
# función cálculo R2, RMSE y MSE
def R2_RMSE(y_true, y_pred):
  # cálculo de R2 y RMSE
  r2 = r2_score(y_true, y_pred)
  mse = mean_squared_error(y_true, y_pred)
  rmse = np.sqrt(mse)
  
  return print('R2:',r2), print('RMSE:',rmse), print('MSE:', mse)

## **_1. Lectura (carga) de los Datos_**

In [3]:
# importamos archivo
df = pd.read_csv('../Data/df_final_RNN.csv')

In [4]:
# miramos el dataframe
df.head()

,year_2023,year_2024,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,...,CAT 798-AC,KOM-CHQ,KOM.960E5,Kom.930E-4,Kom.930E-4SE,Kom.930EN,Kom.930ER,Kom.930ER-MT,Kom.980E5,Komatsu AHS
0,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,24.0,96.0,395.0,96.0,240.0,0.0,132.0,24.0
1,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,24.0,96.0,396.0,96.0,240.0,0.0,132.0,24.0
2,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,24.0,96.0,394.0,96.0,240.0,0.0,132.0,24.0
3,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,24.0,95.0,396.0,96.0,240.0,0.0,132.0,24.0
4,False,False,False,False,False,False,False,False,False,False,...,0.0,0.0,24.0,96.0,395.0,96.0,240.0,0.0,132.0,24.0


In [5]:
# información general df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Data columns (total 85 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year_2023            1567 non-null   bool   
 1   year_2024            1567 non-null   bool   
 2   month_2              1567 non-null   bool   
 3   month_3              1567 non-null   bool   
 4   month_4              1567 non-null   bool   
 5   month_5              1567 non-null   bool   
 6   month_6              1567 non-null   bool   
 7   month_7              1567 non-null   bool   
 8   month_8              1567 non-null   bool   
 9   month_9              1567 non-null   bool   
 10  month_10             1567 non-null   bool   
 11  month_11             1567 non-null   bool   
 12  month_12             1567 non-null   bool   
 13  day_2                1567 non-null   bool   
 14  day_3                1567 non-null   bool   
 15  day_4                1567 non-null   b

## **_2 Preparación del DataFrame_**

In [6]:
# transformamos las variables bool a categorical y le asignamos 0 o 1
for col in df.columns:
  if df[col].dtype == 'bool':
     df[col] = df[col].astype('category').cat.codes
     df[col] = df[col].astype('category') # para que quede como category y no como int8

In [7]:
print('Nombre Columnas: ',df.columns)
print('Cantidad Columnas: ',len(df.columns))

Nombre Columnas:  Index(['year_2023', 'year_2024', 'month_2', 'month_3', 'month_4', 'month_5',
       'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11',
       'month_12', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'day_7',
       'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13', 'day_14',
       'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20', 'day_21',
       'day_22', 'day_23', 'day_24', 'day_25', 'day_26', 'day_27', 'day_28',
       'day_29', 'day_30', 'day_31', 'shift_B', 'crew_E2', 'crew_E3',
       'crew_E4', 'total', 'efectivo', 'mantencion', 'mecanica', 'electrica',
       'mant_excl', 'mec_excl', 'elec_excl', 'som_excl', 'demora', 'r_prog',
       'r_noprg', 'waiting', 'queued', 'material_tonnage', 'payload',
       'distance', 'distance_down', 'distance_down_level', 'distance_up',
       'distance_up_level', 'expected_time', 'lift_down', 'lift_down_level',
       'lift_up', 'lift_up_level', 'travel_time', 'CAT 797-F', 'CAT 798-AC',
     

In [8]:
# cálculamos la desviaicón estándar y la media de payload
std_y = df['payload'].std()
mean_y = df['payload'].mean()

print('Desviación Estándar:',std_y)
print('Media:',mean_y)

Desviación Estándar: 19936.05577427206
Media: 142357.4639507874


In [9]:
# estandarizamos las variables numéricas
col_num = df.drop('payload', axis=1).select_dtypes(include=['float64','int64']).columns

scaler = StandardScaler()
df[col_num] = scaler.fit_transform(df[col_num])

## **_3 XGBoost_**

In [10]:
# definimos el tamaño del set de test
test_size = 0.40

In [11]:
# separamos los datos en X e y
X = df.drop('payload', axis=1)
y = df['payload']

# dividimos los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42) 

In [12]:
# Crear el modelo XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', 
                             enable_categorical=True,
                             random_state=42)

In [13]:
# Entrenar el modelo
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [14]:
# Realizar predicciones en el conjunto de entrenamiento
predictions_train = xgb_model.predict(X_train)

In [15]:
# resultados de R2, RMSE y MSE
R2_RMSE(y_train, predictions_train)

R2: 0.9999453451616311
RMSE: 148.44640361225305
MSE: 22036.33474541194


(None, None, None)

In [16]:
# Realizar predicciones con el modelo entrenado
predictions_test = xgb_model.predict(X_test)

In [17]:
# resultado de R2, RMSE y MSE
R2_RMSE(y_test, predictions_test)

R2: 0.7543477521199697
RMSE: 9764.770308007728
MSE: 95350739.16814935


(None, None, None)

## _4 Optimización del Modelo_ 

### _4.1 Selección de Variables (LassoCV)_

In [18]:
# buscamos alpha optimo con LassoCV
alpha_values = np.linspace(0.01, 300, 30000)

# Crea un objeto LassoCV
lasso_cv = LassoCV(alphas=alpha_values, cv=5)

# Ajusta el modelo a los datos de entrenamiento
lasso_cv.fit(X_train, y_train)

# Valor óptimo de alpha
best_alpha = lasso_cv.alpha_

# Evalúa el modelo en el conjunto de prueba
score = lasso_cv.score(X_test, y_test)

print(f"Mejor valor de alpha: {best_alpha}")
print(f"Puntuación en el conjunto de prueba: {score}")

/home/evansdl/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29762946.630699158, tolerance: 29751362.65422229
  model = cd_fast.enet_coordinate_descent_gram(
/home/evansdl/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29809965.726104736, tolerance: 29751362.65422229
  model = cd_fast.enet_coordinate_descent_gram(
/home/evansdl/miniconda3/envs/py312/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29877489.38156128, tolerance: 29751362.65422229
  model = cd_fast.enet_coordinate_descent_gram(
/home/evansdl/miniconda3/e

Mejor valor de alpha: 56.089999999999996
Puntuación en el conjunto de prueba: 0.8066467716870184


In [19]:
# Graficar MSE vs. Alpha
plt.figure(figsize=(10,6))
plt.plot(lasso_cv.alphas_, np.mean(lasso_cv.mse_path_, axis=1), '-o')
plt.xlabel('Alpha')
plt.ylabel('Mean MSE across CV folds')
plt.title('Path de Regularización')
plt.axvline(lasso_cv.alpha_, color='red', linestyle='--')

# Anotar el valor óptimo de alpha en el gráfico
plt.annotate(f'Optimal alpha: {lasso_cv.alpha_:.2f}, MSE: {np.min(np.mean(lasso_cv.mse_path_, axis=1)):.2f}',
             xy=(lasso_cv.alpha_, np.min(np.mean(lasso_cv.mse_path_, axis=1))),
             xycoords='data', xytext=(10,100), 
             textcoords='offset points',
             arrowprops=dict(arrowstyle="->", lw=1.5),
             fontsize=9)

plt.show()

In [20]:
# Aplicar Lasso para la selección de características
lasso = Lasso(alpha=lasso_cv.alpha_)  
lasso.fit(X_train, y_train)

# Seleccionar las características que tienen coeficientes no nulos
selected_features = X_train.columns[lasso.coef_ != 0]

print("Características seleccionadas:")
print(selected_features)


Características seleccionadas:
Index(['year_2024', 'month_3', 'month_5', 'month_6', 'month_7', 'month_8',
       'month_9', 'month_10', 'month_11', 'day_2', 'day_5', 'day_10', 'day_12',
       'day_14', 'day_22', 'day_25', 'day_26', 'shift_B', 'crew_E2', 'crew_E3',
       'crew_E4', 'total', 'efectivo', 'mecanica', 'electrica', 'mant_excl',
       'mec_excl', 'elec_excl', 'som_excl', 'demora', 'r_noprg', 'waiting',
       'queued', 'material_tonnage', 'distance_down', 'distance_down_level',
       'distance_up', 'distance_up_level', 'expected_time', 'lift_down',
       'lift_up', 'lift_up_level', 'travel_time', 'CAT 797-F', 'CAT 798-AC',
       'KOM.960E5', 'Kom.930E-4', 'Kom.930EN', 'Kom.930ER', 'Kom.980E5',
       'Komatsu AHS'],
      dtype='object')


In [21]:
len(selected_features)

51

> Volvemos ajustar el modelo con las variables seleccionadas

In [22]:
# separamos los datos en train y test
X_lasso= df[selected_features]
y_lasso = df['payload']

# dividimos los datos en train y test
X_train_lasso, X_test_lasso, y_train_lasso, y_test_lasso = train_test_split(X_lasso, y_lasso, test_size=test_size, random_state=42)

In [23]:
# Inicializar el regresor AdaBoost
xgb_model_lasso = xgb.XGBRegressor(objective='reg:squarederror', 
                                   enable_categorical=True,
                                   random_state=42)

In [24]:
# Entrenar el modelo AdaBoost
xgb_model_lasso.fit(X_train_lasso, y_train_lasso)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

#### 4.1.1 Predicciones en el conjunto de entrenamiento

In [25]:
# Realizar predicciones con el modelo entrenado
predictions_lasso_train = xgb_model_lasso.predict(X_train_lasso)

In [26]:
# Evaluar la precisión del modelo
mse = mean_squared_error(y_train_lasso, predictions_lasso_train)
print(f'Mean Squared Error of AdaBoost Regressor: {mse:.2f}')

Mean Squared Error of AdaBoost Regressor: 37244.34


In [27]:
# resultado de R2, RMSE y MSE
R2_RMSE(y_train_lasso, predictions_lasso_train)

R2: 0.9999076260455894
RMSE: 192.98792700273677
MSE: 37244.339968813656


(None, None, None)

#### 4.1.2 Predicciones en el conjunto de testeo

In [28]:
# Realizar predicciones con el modelo entrenado
predictions_lasso_test = xgb_model_lasso.predict(X_test_lasso)

In [29]:
# Evaluar la precisión del modelo
mse = mean_squared_error(y_test_lasso, predictions_lasso_test)
print(f'Mean Squared Error of AdaBoost Regressor: {mse:.2f}')

Mean Squared Error of AdaBoost Regressor: 91126626.43


In [30]:
# resultado de R2, RMSE y MSE
R2_RMSE(y_test_lasso, predictions_lasso_test)

R2: 0.7652303399023671
RMSE: 9546.026735310708
MSE: 91126626.43126681


(None, None, None)

### _4.2 Optimización de Hiperparámetros_

In [31]:
# Definir los hiperparámetros para la búsqueda
param_dist_xgb = {
    'n_estimators': randint(50, 500),
    'learning_rate': uniform(0.01, 0.6),
    'subsample': uniform(0.3, 0.7),
    'max_depth': randint(1, 20),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': uniform(0, 0.5),
    'reg_alpha': uniform(0, 1),  # L1 regularization term
    'reg_lambda': uniform(1, 4)  # L2 regularization term
}

# Crear el modelo XGBoost
xgb_model_hip= xgb.XGBRegressor(objective='reg:squarederror',
                             enable_categorical=True,
                             random_state=42)

# Crear la búsqueda de hiperparámetros
random_search_xgb = RandomizedSearchCV(
    estimator=xgb_model_hip,
    param_distributions=param_dist_xgb,
    n_iter=100,
    cv=5,
    scoring='r2',
    random_state=42,
    n_jobs=-1
)

# Ajustar el modelo
random_search_xgb.fit(X_train_lasso, y_train_lasso)

# Obtener los mejores hiperparámetros
best_params_xgb = random_search_xgb.best_params_
print("Best parameters for XGBoost: ", best_params_xgb)


Best parameters for XGBoost:  {'colsample_bytree': 0.5316874852363839, 'gamma': 0.4949801161949726, 'learning_rate': 0.20341230698483379, 'max_depth': 2, 'n_estimators': 133, 'reg_alpha': 0.25464065476376385, 'reg_lambda': 3.726010888895717, 'subsample': 0.8321595019227805}


In [32]:
# Crear el modelo XGBoost con los mejores hiperparámetros
xgb_model_best = xgb.XGBRegressor(objective='reg:squarederror',
                             enable_categorical=True,
                             random_state=42,
                             **best_params_xgb)

In [33]:
# Entrenar el modelo con los mejores hiperparámetros
xgb_model_best.fit(X_train_lasso, y_train_lasso,
                   eval_set=[(X_test_lasso, y_test_lasso)],
                   early_stopping_rounds=10,
                   verbose=False)

/home/evansdl/miniconda3/envs/py312/lib/python3.12/site-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5316874852363839, device=None,
             early_stopping_rounds=None, enable_categorical=True,
             eval_metric=None, feature_types=None, gamma=0.4949801161949726,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.20341230698483379,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=133, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [34]:
# Realizar predicciones en el conjunto de entrenamiento
train_predictions = xgb_model_best.predict(X_train_lasso)

# Realizar predicciones en el conjunto de prueba
test_predictions = xgb_model_best.predict(X_test_lasso)

In [35]:
# Evaluar el modelo en el conjunto de entrenamiento
train_r2 = r2_score(y_train_lasso, train_predictions)
train_mse = mean_squared_error(y_train_lasso, train_predictions)
train_rmse = np.sqrt(train_mse)

print(f"R2 en el conjunto de entrenamiento: {train_r2:.2f}")
print(f"RMSE en el conjunto de entrenamiento: {train_rmse:.2f}")

R2 en el conjunto de entrenamiento: 0.87
RMSE en el conjunto de entrenamiento: 7255.82


In [36]:
# Evaluar el modelo en el conjunto de prueba
test_r2 = r2_score(y_test_lasso, test_predictions)
test_mse = mean_squared_error(y_test_lasso, test_predictions)
test_rmse = np.sqrt(test_mse)

print(f"R2 en el conjunto de prueba: {test_r2:.2f}")
print(f"RMSE en el conjunto de prueba: {test_rmse:.2f}")

R2 en el conjunto de prueba: 0.80
RMSE en el conjunto de prueba: 8778.51
